In [1]:
import pandas as pd

import numpy as np

import matplotlib.pyplot as plt

In [2]:
df= pd.read_table('./Pre_processed_data/merged_dataframe_shopping.csv', delimiter= ',', encoding='euc-kr')

In [3]:
df.head()

,Unnamed: 0,ID,RCT_NO,BIZ_UNIT,PD_S_C,BR_C,BUY_AM,BUY_CT,AGE_PRD,HOM_PST_NO,...,PD_M_NM,PD_H_NM,IS_MALE,IS_FEMALE,YEAR,MONTH,DAY,HOUR,PRVIN,DESTRICT
0,0,4008,2108,A01,215,2,59600,2,50,056,...,캐주얼,여성의류,0,1,2015,2,16,13,서울특별시,송파구
1,1,6379,2109,A01,75,29,35000,1,40,480,...,건강식품,식품,1,0,2015,2,13,11,부산광역시,해운대구
2,2,6379,2109,A01,149,4,85000,1,40,480,...,화장품,잡화,1,0,2015,1,15,10,부산광역시,해운대구
3,3,8002,2110,A01,138,10,25000,1,30,300,...,섬유잡화,잡화,0,1,2015,12,20,10,세종특별자치시,NaN
4,4,8002,2110,A01,138,10,21000,1,30,300,...,섬유잡화,잡화,0,1,2015,12,20,10,세종특별자치시,NaN


In [4]:
df.drop('Unnamed: 0', axis= 1, inplace= True)

In [5]:
df['YEARMONTH']= 100*df['YEAR']+df['MONTH']

In [6]:
df['DAYHOUR']= 100*df['DAY']+ df['HOUR']

In [7]:
df.drop(['YEAR', 'MONTH', 'DAY', 'HOUR'], axis= 1, inplace= True)

In [8]:
df= pd.get_dummies(df, columns= ['BIZ_UNIT'])

In [58]:
df_201412= df[df['YEARMONTH']== 201412][['ID', 'RCT_NO', 'PD_S_C', 'DAYHOUR', 'BUY_AM', 'BIZ_UNIT_A01', 'BIZ_UNIT_A02', 'BIZ_UNIT_A03', 'BIZ_UNIT_A04', 'BIZ_UNIT_A05']]

In [59]:
agg_func= {'RCT_NO': 'count', 'PD_S_C': lambda x: x.nunique(), 'DAYHOUR': lambda x: x.nunique(), 'BUY_AM': 'sum', 'BIZ_UNIT_A01': 'sum', 'BIZ_UNIT_A02': 'sum', 'BIZ_UNIT_A03': 'sum', 'BIZ_UNIT_A04': 'sum', 'BIZ_UNIT_A05': 'sum',}

In [60]:
df_201412= df_201412.groupby(['ID']).agg(agg_func)

In [66]:
df_merge.head()

,ID,RCT_NO_201412,PD_S_C_201412,DAYHOUR_201412,BUY_AM_201412,BIZ_UNIT_A01_201412,BIZ_UNIT_A02_201412,BIZ_UNIT_A03_201412,BIZ_UNIT_A04_201412,BIZ_UNIT_A05_201412
0,2,6,6,1,17550,0,6,0,0,0
1,6,6,6,1,37860,0,6,0,0,0
2,16,14,12,2,77100,0,14,0,0,0
3,21,11,9,1,33930,0,11,0,0,0
4,31,1,1,1,9100,0,1,0,0,0


In [62]:
df_201412.reset_index(drop= False, inplace= True)

In [63]:
df_201412.columns= df_201412.columns.map(lambda x: x+'_'+'201412')

In [64]:
df_201412.rename(columns= {'ID_201412': 'ID'}, inplace= True)

In [65]:
df_merge= df_201412

In [67]:
YM_list= sorted(df['YEARMONTH'].unique().tolist()[:-1])

In [68]:
for i in YM_list:
    df_toadd= df[df['YEARMONTH']== i][['ID', 'RCT_NO', 'PD_S_C', 'DAYHOUR', 'BUY_AM', 'BIZ_UNIT_A01', 'BIZ_UNIT_A02', 'BIZ_UNIT_A03', 'BIZ_UNIT_A04', 'BIZ_UNIT_A05']]
    df_toadd= df_toadd.groupby(['ID']).agg(agg_func)
    df_toadd.reset_index(drop= False, inplace= True)
    df_toadd.columns= df_toadd.columns.map(lambda x: x+'_'+str(i))
    idname= 'ID_'+str(i)
    df_toadd.rename(columns= {idname: 'ID'}, inplace= True)
    df_merge= pd.merge(df_merge, df_toadd, how= 'outer', on= 'ID')
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18550 entries, 0 to 18549
Columns: 118 entries, ID to BIZ_UNIT_A05_201512
dtypes: float64(117), int64(1)
memory usage: 16.8 MB


In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3816952 entries, 0 to 3816951
Data columns (total 22 columns):
ID              int64
RCT_NO          int64
PD_S_C          int64
BR_C            int64
BUY_AM          int64
BUY_CT          int64
AGE_PRD         int64
HOM_PST_NO      object
PD_S_NM         object
PD_M_NM         object
PD_H_NM         object
IS_MALE         int64
IS_FEMALE       int64
PRVIN           object
DESTRICT        object
YEARMONTH       int64
DAYHOUR         int64
BIZ_UNIT_A01    uint8
BIZ_UNIT_A02    uint8
BIZ_UNIT_A03    uint8
BIZ_UNIT_A04    uint8
BIZ_UNIT_A05    uint8
dtypes: int64(11), object(6), uint8(5)
memory usage: 513.3+ MB


In [73]:
df_merge[df_merge['ID']<30]

,ID,RCT_NO_201412,PD_S_C_201412,DAYHOUR_201412,BUY_AM_201412,BIZ_UNIT_A01_201412,BIZ_UNIT_A02_201412,BIZ_UNIT_A03_201412,BIZ_UNIT_A04_201412,BIZ_UNIT_A05_201412,...,BIZ_UNIT_A05_201511,RCT_NO_201512,PD_S_C_201512,DAYHOUR_201512,BUY_AM_201512,BIZ_UNIT_A01_201512,BIZ_UNIT_A02_201512,BIZ_UNIT_A03_201512,BIZ_UNIT_A04_201512,BIZ_UNIT_A05_201512
0,2,6.0,6.0,1.0,17550.0,0.0,6.0,0.0,0.0,0.0,...,0.0,13.0,8.0,2.0,58640.0,0.0,13.0,0.0,0.0,0.0
1,6,6.0,6.0,1.0,37860.0,0.0,6.0,0.0,0.0,0.0,...,0.0,58.0,44.0,17.0,505373.0,0.0,41.0,7.0,10.0,0.0
2,16,14.0,12.0,2.0,77100.0,0.0,14.0,0.0,0.0,0.0,...,0.0,45.0,37.0,19.0,1590350.0,12.0,28.0,5.0,0.0,0.0
3,21,11.0,9.0,1.0,33930.0,0.0,11.0,0.0,0.0,0.0,...,0.0,94.0,40.0,20.0,786610.0,16.0,78.0,0.0,0.0,0.0
1819,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,6.0,5.0,3.0,227050.0,6.0,0.0,0.0,0.0,0.0
1820,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,19.0,13.0,6.0,401870.0,9.0,8.0,2.0,0.0,0.0
1821,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,23.0,16.0,7.0,237430.0,8.0,15.0,0.0,0.0,0.0
1822,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,2.0,1.0,1.0,23200.0,2.0,0.0,0.0,0.0,0.0
1823,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,45.0,27.0,1.0,225590.0,0.0,45.0,0.0,0.0,0.0
1824,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,25.0,17.0,8.0,841370.0,11.0,0.0,14.0,0.0,0.0


In [75]:
df_merge.fillna(0, inplace= True)

In [76]:
df_merge.to_csv('groupby_YEARMONTH_columnwise.csv')